# ETL Google.
Vamos a adaptar las tablas de google al esquema E/R definido y cargar las tablas a BigQuery usando el connector respectivo.

In [212]:
import os

# SPARK trabaja con hadoop, se especifica los directorios de instalacion
os.environ["SPARK_HOME"] = r"F:\DataScience\spark\spark-3.5.3-bin-hadoop3\spark-3.5.3-bin-hadoop3"
os.environ["HADOOP_HOME"] = r"F:\DataScience\hadoop\hadoop-3.4.0"
os.environ["HADOOP_CONF_DIR"] = r"F:\DataScience\hadoop\hadoop-3.4.0\etc\hadoop"

import findspark
import pickle
import pandas as pd
import numpy as np

findspark.init()


# Asignación de recursos locales para SPARK.
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[10]")\
    .config("spark.executor.memory", "24g")\
    .config("spark.driver.memory", "6g")\
    .config("spark.sql.shuffle.partitions", "30")\
    .config('spark.jars', r'F:\DataScience\PF - DataNova\JARs\spark-3.5-bigquery-0.41.1.jar')\
    .appName("PySpark").getOrCreate()

In [260]:
# Funciones de spark
from pyspark.sql import functions as F

# Funciones para graficar.
import seaborn as sns
import matplotlib.pyplot as plt

# Tabla Metadata

In [3]:
metadata_df = spark.read.json(r'F:\DataScience\PF - DataNova\datasets\Google Maps\metadata-sitios\*.json')
metadata_df_cached = metadata_df.cache()

In [4]:
food_keywords = ['coffee', 'espresso', 'bistro', 'cafe\\\\b', 'roaster' ]  # Se usara cafe\\\\b para evitiar matches con Cafeterias. 
regex_pattern = '|'.join(food_keywords)

In [5]:
df_filtered = metadata_df_cached.filter(
    F.expr(f"array_contains(transform(category, x -> lower(x) rlike '{regex_pattern}'), true)")
)

df_filtered.show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+-------------------------------------+--------------------

In [106]:
df_filtered.printSchema()

root
 |-- MISC: struct (nullable = true)
 |    |-- Accessibility: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Activities: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Amenities: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Atmosphere: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Crowd: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Dining options: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- From the business: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Getting here: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Health & safety: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Health and safety: array (nullable = true)


In [6]:
locations_df = df_filtered.select('gmap_id','latitude','longitude').toPandas()

In [7]:
locations_df.head(5)

,gmap_id,latitude,longitude
0,0x89c6c715d1821fe3:0x9cfa8308c0ce2289,39.950078,-75.180729
1,0x7954d370921ff6bd:0x3193ba783e26d032,20.888238,-156.450614
2,0x87faba1e67ff3e8d:0xdb90d5f5c33c5a6e,42.701941,-92.225543
3,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,36.099575,-115.176338
4,0x89ac5f9990c10c4b:0xd5a96da0fd76a3e3,35.776012,-78.610680


Vamos a agregar la ciudad y el estado, ya que es parte del esquema de nuestra tabla.

# Estado

In [8]:
# Vamos a usar un shapefile para esta tarea.
import geopandas as gpd
from shapely.geometry import Point

In [9]:
# Se lee el archivo shp
states = gpd.read_file(r"F:\DataScience\PF - DataNova\datasets\External Data\shapefile\cb_2018_us_state_500k\cb_2018_us_state_500k.shp") 

In [10]:
# Visualizacion
states.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,28,01779790,0400000US28,28,MS,Mississippi,00,121533519481,3926919758,"MULTIPOLYGON (((-88.50297 30.21524, -88.49176 ..."
1,37,01027616,0400000US37,37,NC,North Carolina,00,125923656064,13466071395,"MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ..."
2,40,01102857,0400000US40,40,OK,Oklahoma,00,177662925723,3374587997,"POLYGON ((-103.00256 36.52659, -103.00219 36.6..."
3,51,01779803,0400000US51,51,VA,Virginia,00,102257717110,8528531774,"MULTIPOLYGON (((-75.74241 37.80835, -75.74151 ..."
4,54,01779805,0400000US54,54,WV,West Virginia,00,62266474513,489028543,"POLYGON ((-82.6432 38.16909, -82.643 38.16956,..."


In [11]:
# Aqui vamos a hacer el cruce de información.
geometry = [Point(lon, lat) for lon, lat in zip(locations_df['longitude'], locations_df['latitude'])]
gdf_puntos = gpd.GeoDataFrame(locations_df, geometry=geometry)

gdf_puntos.crs = "EPSG:4326"  # WGS84
states = states.to_crs("EPSG:4326")  

gdf_joined = gpd.sjoin(gdf_puntos, states, how="left")

gdf_joined

,gmap_id,latitude,longitude,geometry,index_right,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER
0,0x89c6c715d1821fe3:0x9cfa8308c0ce2289,39.950078,-75.180729,POINT (-75.18073 39.95008),19.0,42,01779798,0400000US42,42,PA,Pennsylvania,00,1.158844e+11,3.394590e+09
1,0x7954d370921ff6bd:0x3193ba783e26d032,20.888238,-156.450614,POINT (-156.45061 20.88824),42.0,15,01779782,0400000US15,15,HI,Hawaii,00,1.663399e+10,1.177781e+10
2,0x87faba1e67ff3e8d:0xdb90d5f5c33c5a6e,42.701941,-92.225543,POINT (-92.22554 42.70194),32.0,19,01779785,0400000US19,19,IA,Iowa,00,1.446613e+11,1.084181e+09
3,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,36.099575,-115.176338,POINT (-115.17634 36.09958),28.0,32,01779793,0400000US32,32,NV,Nevada,00,2.843295e+11,2.047206e+09
4,0x89ac5f9990c10c4b:0xd5a96da0fd76a3e3,35.776012,-78.610680,POINT (-78.61068 35.77601),1.0,37,01027616,0400000US37,37,NC,North Carolina,00,1.259237e+11,1.346607e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43153,0x54011e1ea303d8fb:0x6926865e295a2860,58.366200,-134.591000,POINT (-134.591 58.3662),27.0,02,01785533,0400000US02,02,AK,Alaska,00,1.478840e+12,2.454816e+11
43154,0x89c25af6e18e56a5:0x9b38fce3dbd88098,40.674726,-73.997918,POINT (-73.99792 40.67473),25.0,36,01779796,0400000US36,36,NY,New York,00,1.220491e+11,1.924699e+10
43155,0x874108492da42871:0xffdd87f3eeced9c2,39.520075,-107.319044,POINT (-107.31904 39.52007),21.0,08,01779779,0400000US08,08,CO,Colorado,00,2.684229e+11,1.181622e+09
43156,0x80dcd91a30eeb005:0xda3db9bb93ef4937,33.734955,-117.868623,POINT (-117.86862 33.73495),16.0,06,01779778,0400000US06,06,CA,California,00,4.035039e+11,2.046387e+10


In [12]:
# Seleccionamos solamente las columnas necesarias.
locations_df = gdf_joined[['gmap_id', 'latitude', 'longitude', 'NAME']]
locations_df.rename(columns={'NAME':'state'}, inplace=True)
locations_df.head(5)

C:\Users\Une\AppData\Local\Temp\ipykernel_24848\3467693935.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_df.rename(columns={'NAME':'state'}, inplace=True)


,gmap_id,latitude,longitude,state
0,0x89c6c715d1821fe3:0x9cfa8308c0ce2289,39.950078,-75.180729,Pennsylvania
1,0x7954d370921ff6bd:0x3193ba783e26d032,20.888238,-156.450614,Hawaii
2,0x87faba1e67ff3e8d:0xdb90d5f5c33c5a6e,42.701941,-92.225543,Iowa
3,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,36.099575,-115.176338,Nevada
4,0x89ac5f9990c10c4b:0xd5a96da0fd76a3e3,35.776012,-78.610680,North Carolina


# Ciudad

In [ ]:
# Leemos y concatenamos todos los SHP files.
shp_path  = r"F:\DataScience\PF - DataNova\datasets\External Data\shapefile\TIGER"

iter_counter = 0
for file in os.listdir(shp_path):
    if file.endswith('.shp'):

        file_path = shp_path + "\\" + file
        cities = gpd.read_file(file_path)

        if iter_counter==0:
            iter_counter+=1
            cities_geo = cities.copy()
        else:
            cities_geo = pd.concat([cities_geo, cities], ignore_index=True)



In [76]:
# Realizamos nuestro cruce con los locales obtenidos.
geometry = [Point(lon, lat) for lon, lat in zip(locations_df['longitude'], locations_df['latitude'])]
gdf_puntos = gpd.GeoDataFrame(locations_df, geometry=geometry)
gdf_puntos.crs = "EPSG:4326"  # WGS84
cities = cities_geo.to_crs("EPSG:4326")  
gdf_joined = gpd.sjoin(gdf_puntos, cities, how="left")

In [79]:
gdf_joined.columns

Index(['gmap_id', 'latitude', 'longitude', 'state', 'geometry', 'index_right',
       'STATEFP', 'PLACEFP', 'PLACENS', 'GEOID', 'GEOIDFQ', 'NAME', 'NAMELSAD',
       'LSAD', 'CLASSFP', 'PCICBSA', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER',
       'INTPTLAT', 'INTPTLON'],
      dtype='object')

In [80]:
locations_df = gdf_joined[['gmap_id', 'state', 'latitude', 'longitude', 'NAME']]
locations_df.rename(columns={'NAME':'city'}, inplace=True)
locations_df.head(5)

C:\Users\Une\AppData\Local\Temp\ipykernel_24848\2257291677.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_df.rename(columns={'NAME':'city'}, inplace=True)


,gmap_id,state,latitude,longitude,city
0,0x89c6c715d1821fe3:0x9cfa8308c0ce2289,Pennsylvania,39.950078,-75.180729,Philadelphia
1,0x7954d370921ff6bd:0x3193ba783e26d032,Hawaii,20.888238,-156.450614,Kahului
2,0x87faba1e67ff3e8d:0xdb90d5f5c33c5a6e,Iowa,42.701941,-92.225543,Readlyn
3,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Nevada,36.099575,-115.176338,Paradise
4,0x89ac5f9990c10c4b:0xd5a96da0fd76a3e3,North Carolina,35.776012,-78.610680,Raleigh


# Remoción de vacíos

In [86]:
locations_df.dropna(inplace=True)
locations_df.reset_index(drop=True, inplace=True)

C:\Users\Une\AppData\Local\Temp\ipykernel_24848\1444285571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_df.dropna(inplace=True)


In [87]:
locations_df

,gmap_id,state,latitude,longitude,city
0,0x89c6c715d1821fe3:0x9cfa8308c0ce2289,Pennsylvania,39.950078,-75.180729,Philadelphia
1,0x7954d370921ff6bd:0x3193ba783e26d032,Hawaii,20.888238,-156.450614,Kahului
2,0x87faba1e67ff3e8d:0xdb90d5f5c33c5a6e,Iowa,42.701941,-92.225543,Readlyn
3,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Nevada,36.099575,-115.176338,Paradise
4,0x89ac5f9990c10c4b:0xd5a96da0fd76a3e3,North Carolina,35.776012,-78.610680,Raleigh
...,...,...,...,...,...
38411,0x54011e1ea303d8fb:0x6926865e295a2860,Alaska,58.366200,-134.591000,Juneau
38412,0x89c25af6e18e56a5:0x9b38fce3dbd88098,New York,40.674726,-73.997918,New York
38413,0x874108492da42871:0xffdd87f3eeced9c2,Colorado,39.520075,-107.319044,Glenwood Springs
38414,0x80dcd91a30eeb005:0xda3db9bb93ef4937,California,33.734955,-117.868623,Santa Ana


# Cruce de datos

In [ ]:
# Extramos las columnas necesarias
business_df = df_filtered.select('gmap_id', 'name', 'address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews', 'MISC', 'category').toPandas()
business_df.head()

,gmap_id,name,address,latitude,longitude,avg_rating,num_of_reviews,MISC,category
0,0x89c6c715d1821fe3:0x9cfa8308c0ce2289,Cadence Cycling and Cafe - Center City,"Cadence Cycling and Cafe - Center City, 201 S ...",39.950078,-75.180729,4.8,38,"([Wheelchair accessible entrance], None, None,...","[Bicycle Shop, Bicycle repair shop, Coffee sho..."
1,0x7954d370921ff6bd:0x3193ba783e26d032,Kraken Coffee Kahului,"Kraken Coffee Kahului, 520 Keolani Pl, Kahului...",20.888238,-156.450614,4.8,8,"(None, None, None, [Casual], [College students...",[Coffee shop]
2,0x87faba1e67ff3e8d:0xdb90d5f5c33c5a6e,The Center Inn,"The Center Inn, 209 Main St, Readlyn, IA 50668",42.701941,-92.225543,4.5,15,"([Wheelchair accessible entrance], None, None,...",[Restaurant or cafe]
3,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Castle Coffee,"Castle Coffee, Excalibur Hotel and Casino, 385...",36.099575,-115.176338,3.2,24,"([Wheelchair accessible entrance, Wheelchair a...",[Coffee shop]
4,0x89ac5f9990c10c4b:0xd5a96da0fd76a3e3,Blue Dragon Sweepstakes,"Blue Dragon Sweepstakes, 2008 Poole Rd, Raleig...",35.776012,-78.610680,4.2,33,"([Wheelchair accessible entrance, Wheelchair a...",[Internet cafe]


In [98]:
merged_df = pd.merge(left=business_df, right=locations_df[['gmap_id', 'city', 'state']], how='inner', on='gmap_id')[['gmap_id', 'name', 'address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews',
                                                                           'category', 'state', 'city', 'MISC']]

In [107]:
merged_df.columns

Index(['gmap_id', 'name', 'address', 'latitude', 'longitude', 'avg_rating',
       'num_of_reviews', 'category', 'state', 'city', 'MISC'],
      dtype='object')

In [223]:
merged_df.to_parquet(r'F:\DataScience\PF - DataNova\ETL\ETL-Datasets\business_google.parquet')

# Carga a Bigquery

### Tabla business

In [108]:
# Acomodamos los datos al esquema.
renamer = {'gmap_id' : 'business_id',
           'avg_rating' : 'business_star',
           'num_of_reviews' : 'review_count',
           'category' : 'categories'}

merged_df.rename(columns=renamer, inplace=True)

In [110]:
merged_df.columns

Index(['business_id', 'name', 'address', 'latitude', 'longitude',
       'business_star', 'review_count', 'categories', 'state', 'city', 'MISC'],
      dtype='object')

In [111]:
business_df = merged_df[['business_id', 'name', 'address', 'city', 'state', 'business_star','latitude', 'longitude', 'categories',
        'review_count']]

In [112]:
business_df['app_source'] = 'Google'

In [115]:
business_df = business_df[['business_id', 'name', 'address', 'city', 'state', 'business_star','latitude', 'longitude', 'app_source', 'categories',
        'review_count']]

In [146]:
# Definimos la ruta de environment donde guardamos nuestra credencial de GCP.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"F:\DataScience\PF - DataNova\gcp key\bamboo-zone-445202-a3-be5b705efba8.json"

In [102]:
from google.cloud import bigquery

In [121]:
project_id = "bamboo-zone-445202-a3"
dataset_id = "Datanova_CoffeeShops"
table_id = "business"

In [150]:
client = bigquery.Client()

# Define el nombre completo de la tabla
table_full_id = f"{project_id}.{dataset_id}.{table_id}"

In [151]:
job = client.load_table_from_dataframe(
    business_df,  # Tu DataFrame
    table_full_id,
    job_config=bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",
        autodetect=False  # Sobrescribe si la tabla ya existe
    )
)

job.result()

LoadJob<project=bamboo-zone-445202-a3, location=US, id=a5c4db40-3083-4a78-af78-88f97525bfc6>

In [187]:
business_df.loc[0, 'categories']

list

### Tabla Google_misc

In [290]:
misc_df = merged_df[['business_id', 'MISC']]

In [291]:
def list_maker(row):
    try:
        return list(row)
    except:
        return []

In [298]:
misc_df['MISC'] = misc_df['MISC'].apply(str)

C:\Users\Une\AppData\Local\Temp\ipykernel_24848\819243773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misc_df['MISC'] = misc_df['MISC'].apply(str)


In [301]:
type(misc_df.iloc[0,1])

str

In [302]:
table_full_id = "bamboo-zone-445202-a3.Datanova_CoffeeShops.google_misc"

In [303]:
job = client.load_table_from_dataframe(
    misc_df,  # Tu DataFrame
    table_full_id,
    job_config=bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",
        autodetect=False  # Sobrescribe si la tabla ya existe
    )
)

job.result()

LoadJob<project=bamboo-zone-445202-a3, location=US, id=ddcfce31-840d-4d9b-a14b-d2f6bea2b05f>

# Dataset reviews

In [231]:
# Leemos el dataset reviews
reviews_df = spark.read.json(r'F:\DataScience\PF - DataNova\datasets\Google Maps\reviews-estados\**\*.json')
reviews_df_cached = reviews_df.cache()

In [235]:
# Revisamos el schema
reviews_df_cached.printSchema()

root
 |-- gmap_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |-- rating: long (nullable = true)
 |-- resp: struct (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- time: long (nullable = true)
 |-- text: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)



In [233]:
# Leemos el dataset de business_df
business_df = spark.read.parquet(r'F:\DataScience\PF - DataNova\ETL\ETL-Datasets\business_google.parquet')
business_df_cached = business_df.cache()

In [236]:
# Verificamos el esquema
business_df_cached.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- business_star: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- MISC: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)



In [ ]:
# Renombramos las columnas
reviews_df_cached = reviews_df_cached.withColumnRenamed("gmap_id", "business_id")
# Realizamos un merge. 
reviews_filtered_df = reviews_df_cached.join(business_df_cached.select('business_id'), on='business_id', how='inner')

In [238]:
# Almacenamos en cache.
reviews_filtered_df_cached = reviews_filtered_df.cache()

In [243]:
# Dropeamos duplicados.
reviews_filtered_df_cached = reviews_filtered_df_cached.dropDuplicates()

In [246]:
# Seleccionamos las columnas que requerimos, y renombramos.
reviews_df = reviews_filtered_df_cached.select('business_id', 'rating', 'text', 'time', 'user_id')
reviews_df = reviews_df.withColumnRenamed('rating', 'stars')
reviews_df = reviews_df.withColumnRenamed('time', 'date')
reviews_df.show()

+--------------------+-----+--------------------+-------------+--------------------+
|         business_id|stars|                text|         date|             user_id|
+--------------------+-----+--------------------+-------------+--------------------+
|0x874fa3d02119e08...|    5|Classy dining are...|1617988434370|11781109703466512...|
|0x87530ede9bbcf81...|    4|                NULL|1580408757173|11474481699655525...|
|0x87530ede9bbcf81...|    2|                NULL|1575244750143|10528073849571460...|
|0x875272bc3684140...|    4|Cute little store...|1570589160899|11462465646238487...|
|0x87526196e3fbe2d...|    5|Coffee was splend...|1628972565979|11773814681191421...|
|0x87526196e3fbe2d...|    3|Fun that this cof...|1629043699731|10208622639739157...|
|0x87525f8cfa0b583...|    3|                NULL|1550375437540|11656403510487355...|
|0x87525f8cfa0b583...|    5|                NULL|1531439036948|11177574547299381...|
|0x87525f8d1b54aac...|    5|Friendly staff, q...|1458666527738|11

In [250]:
# Agregamos un identificador unico para el review.
from pyspark.sql.functions import expr

reviews_df = reviews_df.withColumn("review_id", expr("uuid()"))


In [ ]:
# Agregamos al columna app_source.
from pyspark.sql.functions import lit

reviews_df = reviews_df.withColumn("app_source", lit("Google"))

In [266]:
# Damos formato a la columna date
reviews_df = reviews_df.withColumn("date_column_timestamp", F.from_unixtime(F.col("date") / 1000).cast("timestamp"))
reviews_df = reviews_df.drop('date')
reviews_df = reviews_df.withColumnRenamed('date_column_timestamp','date')
reviews_df.show()


+--------------------+-----+--------------------+--------------------+--------------------+----------+-------------------+
|         business_id|stars|                text|             user_id|           review_id|app_source|               date|
+--------------------+-----+--------------------+--------------------+--------------------+----------+-------------------+
|0x874fa3d02119e08...|    5|Classy dining are...|11781109703466512...|a19970a0-b59b-446...|    Google|2021-04-09 12:13:54|
|0x87530ede9bbcf81...|    4|                NULL|11474481699655525...|85f97315-4a78-457...|    Google|2020-01-30 13:25:57|
|0x87530ede9bbcf81...|    2|                NULL|10528073849571460...|2ab1fe34-aff0-46d...|    Google|2019-12-01 18:59:10|
|0x875272bc3684140...|    4|Cute little store...|11462465646238487...|088ee1cb-d3d1-482...|    Google|2019-10-08 21:46:00|
|0x87526196e3fbe2d...|    5|Coffee was splend...|11773814681191421...|ccdb5803-98a0-404...|    Google|2021-08-14 15:22:45|
|0x87526196e3fbe

In [267]:
reviews_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- review_id: string (nullable = false)
 |-- app_source: string (nullable = false)
 |-- date: timestamp (nullable = true)



In [258]:
reviews_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- date: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- review_id: string (nullable = false)
 |-- app_source: string (nullable = false)



### Carga a BQ

In [270]:
reviews_df_pd = reviews_df.toPandas()

In [273]:
reviews_df_pd

,business_id,stars,text,user_id,review_id,app_source,date
0,0x874fa3d02119e087:0xe4ba10bcfa0d6127,5,"Classy dining area, friendly, professional ser...",117811097034665127052,a19970a0-b59b-446e-9a1a-970f04482e8c,Google,2021-04-09 12:13:54
1,0x87530ede9bbcf81d:0xddefad57493a84de,4,None,114744816996555254159,85f97315-4a78-4572-80fe-84117269c5bb,Google,2020-01-30 13:25:57
2,0x87530ede9bbcf81d:0xddefad57493a84de,2,None,105280738495714606114,2ab1fe34-aff0-46d9-9fff-e8ab9e540190,Google,2019-12-01 18:59:10
3,0x875272bc36841409:0xc5db06be11185f97,4,Cute little store with a lot of eclectic finds...,114624656462384879979,088ee1cb-d3d1-4828-b0e0-a54b4394d591,Google,2019-10-08 21:46:00
4,0x87526196e3fbe2d9:0x18cdf54050d62d90,5,Coffee was splendid and we loved spending time...,117738146811914218396,ccdb5803-98a0-4043-9ce4-68477281d0e9,Google,2021-08-14 15:22:45
...,...,...,...,...,...,...,...
1511202,0x884a67ca224fb353:0xd33fa06a2d3b9000,5,Always excellent!,108706145744307561085,8f227355-4f06-4706-86bc-a71bebf1279e,Google,2019-09-07 19:30:11
1511203,0x884a67ca224fb353:0xd33fa06a2d3b9000,4,None,107907487505808249243,16dc7830-118a-40cb-a804-e3acb789ecd7,Google,2018-01-24 20:41:44
1511204,0x884a67ca224fb353:0xd33fa06a2d3b9000,5,None,114570371641063236705,72d5b14d-fb52-4dad-8026-74fb0f41224f,Google,2019-09-01 10:41:45
1511205,0x8835d98f80ed05e1:0xc4ff258dba24f48c,5,Good price for the food,101321919402153608581,d0f39726-0229-47e6-90ec-167a9c7cc88c,Google,2019-05-27 06:14:49


In [268]:
table_path = "bamboo-zone-445202-a3.Datanova_CoffeeShops.reviews"

In [272]:
job = client.load_table_from_dataframe(
    reviews_df_pd,  # Tu DataFrame
    table_path,
    job_config=bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",
        autodetect=False  # Sobrescribe si la tabla ya existe
    )
)

job.result()

LoadJob<project=bamboo-zone-445202-a3, location=US, id=af8314ae-b4ef-45e5-9b42-78b1b742df74>

# Tabla Users
Google no tiene una tabla users. La extraeremos de reviews

In [305]:
spark.catalog.clearCache()

In [307]:
reviews_filtered_df_cached = reviews_filtered_df_cached.cache()

In [308]:
reviews_filtered_df_cached.show()

+--------------------+------------------+--------------------+------+--------------------+--------------------+-------------+--------------------+
|         business_id|              name|                pics|rating|                resp|                text|         time|             user_id|
+--------------------+------------------+--------------------+------+--------------------+--------------------+-------------+--------------------+
|0x874fa3d02119e08...|    Marlyn Knudsen|                NULL|     5|                NULL|Classy dining are...|1617988434370|11781109703466512...|
|0x87530ede9bbcf81...|    Travis Finster|                NULL|     4|                NULL|                NULL|1580408757173|11474481699655525...|
|0x87530ede9bbcf81...|    Juliana Pulido|                NULL|     2|                NULL|                NULL|1575244750143|10528073849571460...|
|0x875272bc3684140...|       Beth Heyden|                NULL|     4|{Thanks for visit...|Cute little store...|1570589

In [309]:
reviews_filtered_df_cached.count()

1511207

In [312]:
user_df = reviews_filtered_df_cached.groupBy('user_id', 'name').agg(
    F.count('business_id').alias('review_count'),
    F.avg('rating').alias('average_stars'))

In [314]:
user_df.count()

1250070

In [316]:
user_df = user_df.withColumn('app_source', lit('Google'))

In [317]:
user_df.show()

+--------------------+------------------+------------+------------------+----------+
|             user_id|              name|review_count|     average_stars|app_source|
+--------------------+------------------+------------+------------------+----------+
|11264278833015883...|       Anna Lensch|           1|               5.0|    Google|
|11072859995682382...|     Jennifer Dean|           1|               5.0|    Google|
|11730908388943399...|      Bob Borkovec|           3| 4.333333333333333|    Google|
|11697525190344988...|     Teri Hansford|           1|               5.0|    Google|
|10870509505599109...|  Patricia Whitman|           1|               5.0|    Google|
|11078859066613529...|        Queen Lucy|           1|               5.0|    Google|
|11121890834243035...|     Christina New|           2|               5.0|    Google|
|10586996229479852...|   Vanessa Flowers|           1|               5.0|    Google|
|10221721553634831...|   Karen Hernandez|           2|           

In [318]:
user_df_pd = user_df.toPandas()

### Carga a BQ

In [319]:
path= "bamboo-zone-445202-a3.Datanova_CoffeeShops.user"

In [320]:
job = client.load_table_from_dataframe(
    user_df_pd,  # Tu DataFrame
    path,
    job_config=bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",
        autodetect=False  # Sobrescribe si la tabla ya existe
    )
)

job.result()

LoadJob<project=bamboo-zone-445202-a3, location=US, id=ae66a0e0-22b0-4494-ae3e-c76b7ed082e1>

# Pendientes

1. Cargar tabla MISC - DONE
2. Realizar ETL de reviews (cruce con los negocios, mapear las columnas requeridas, y cargar a gcloud.) - DONE
3. Cargar tabla USERs de Google - DONE
3. API de GoogleCloud, proceso de carga automatica (Cloud Functions, Cloud Scheduler) . Pendiente